# 임베딩 생성 및 벡터 저장 실습 (Ko-SBERT 임베딩 모델 사용)

- 사용 모델: jhgan/ko-sroberta-multitask
- 한국어 텍스트에 특화된 SBERT 기반 임베딩 모델
- 범용 문장 임베딩에 널리 사용됨
- 참고 링크: https://huggingface.co/jhgan/ko-sroberta-multitask
- 실습 목표
    - KoSBERT 모델이 어떤 모델인지 이해하고
    - 텍스트 → 벡터화 → 인덱싱 → 검색 테스트까지 진행
    - E5와 비교해 한국어 질문 처리 방식의 차이를 느껴보기

# 1. 필수 라이브러리 설치 및 로드

In [1]:
# !pip install -U langchain faiss-cpu langchain-huggingface

In [2]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 2. Ko-SBERT 임베딩 모델 설명

- 한국어 문장을 벡터로 변환하기 위해 설계된 SBERT 기반 모델
- 다양한 자연어 추론 및 의미 유사도 태스크를 멀티태스크로 학습해, 한국어 문장 간 의미 비교에 효과적

특히 오프라인 환경에서도 실행할 수 있고, HuggingFace에서 무료로 제공되기 때문에 RAG 구조에서 빠른 프로토타이핑에 유용합니다.

# 3. 텍스트 로드 및 문단 단위 분할

In [3]:
# 텍스트 로드
file_path = "kr_constitution_cleaned.txt"

with open(file_path, "r", encoding="utf-8") as f:
    full_text = f.read()

In [4]:
# 문단 단위 분할 (Chunking)
chunks = full_text.split("\n")
chunks = [c.strip() for c in chunks if len(c.strip()) > 30]

print("총 문단 수:", len(chunks))
print("샘플 문단:", chunks[0])

총 문단 수: 261
샘플 문단: [시행 1988. 2. 25.] [헌법 제10호, 1987. 10. 29., 전부개정]


# 5. 임베딩 모델 로드 (Ko-SBERT)

In [5]:
model_name = "jhgan/ko-sroberta-multitask"
embedding = HuggingFaceEmbeddings(model_name=model_name)

/Users/jiho/Desktop/rag-chatbot-guide/rag-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 6. 텍스트 벡터화 및 FAISS 인덱스 생성

In [6]:
faiss_index = FAISS.from_texts(chunks, embedding)
faiss_index.save_local("embeddings/faiss_index_kosbert")
print("\n✅ FAISS 인덱스 저장 완료")


✅ FAISS 인덱스 저장 완료


# 7. 벡터 검색 테스트

In [7]:
from langchain_core.runnables import Runnable

retriever = faiss_index.as_retriever()

# 상위 k개 문서 검색
retriever.search_kwargs["k"] = 5

query = "국회의 임기는 몇 년인가요?"

results = retriever.invoke(query)

print("\n🔍 질문:", query)
for i, doc in enumerate(results):
    print(f"[{i+1}] {doc.page_content.strip()[:1000]}...")


🔍 질문: 국회의 임기는 몇 년인가요?
[1] 제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원 4분의 1 이상의 요구에 의하여 집회된다....
[2] ②이 헌법공포 당시의 국회의원의 임기는 제1항에 의한 국회의 최초의 집회일 전일까지로 한다....
[3] 제51조 국회에 제출된 법률안 기타의 의안은 회기 중에 의결되지 못한 이유로 폐기되지 아니한다. 다만, 국회의원의 임기가 만료된 때에는 그러하지 아니하다....
[4] 제55조 ①한 회계연도를 넘어 계속하여 지출할 필요가 있을 때에는 정부는 연한을 정하여 계속비로서 국회의 의결을 얻어야 한다....
[5] 제3조 ①이 헌법에 의한 최초의 국회의원선거는 이 헌법공포일로부터 6월 이내에 실시하며, 이 헌법에 의하여 선출된 최초의 국회의원의 임기는 국회의원선거후 이 헌법에 의한 국회의 최초의 집회일로부터 개시한다....
